In [1]:
from datasets import load_dataset

In [2]:
raw_datasets = load_dataset('kde4',lang1='en', lang2 = 'fr')

Found cached dataset kde4 (C:/Users/niels/.cache/huggingface/datasets/kde4/en-fr-lang1=en,lang2=fr/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
raw_datasets['train'][4000]

{'id': '4000',
 'translation': {'en': 'To compile your source code for screen viewers like & okular; or further conversion, you can use the shortcut Alt; 2. Then you can view the & DVI; file using your default viewer with Alt; 3, convert the & DVI; to a PS file with Alt; 4, and view the PS file with Alt; 5.',
  'fr': "Pour compiler votre code source pour un afficheur à l'écran avec & okular; ou pour d'autres conversions, vous pouvez utiliser le raccourci Alt; 2. Vous pouvez alors voir le fichier & DVI; en utilisant votre afficheur par défaut avec Alt; 3, convertir le & DVI; en fichier PS avec Alt; 4, et voir le fichier PS avec Alt; 5."}}

In [4]:
split_datasets = raw_datasets['train'].train_test_split(train_size=0.9,seed=20)

Loading cached split indices for dataset at C:\Users\niels\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-496be247a58b47c1.arrow and C:\Users\niels\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-2c0faebb61cdd12e.arrow


In [5]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [7]:
raw_datasets['train'][19]['translation']

{'en': 'You may only translate web pages that are accessible on the world wide web. To translate any other text, you should go directly to the Babelfish site itself, where you are able to paste in text for translation.',
 'fr': "Il n'est possible de traduire que des pages web qui sont accessibles sur l'internet. Pour la traduction de tout autre texte, il est nécessaire de se rendre directement sur le site de Babelfish lui-même, ou il vous est possible de coller du texte à traduire."}

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame.from_records(raw_datasets['train']['translation'])

In [ ]:
df.head()

In [10]:
from transformers import pipeline

In [11]:
model_checkpoint = 'Helsinki-NLP/opus-mt-en-fr'
translator = pipeline('translation',model = model_checkpoint)
translator('Default to expanded threads')

C:\Users\niels\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Par défaut pour les threads élargis'}]

In [12]:
from transformers import AutoTokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,return_tensors='tf')

In [14]:
en_sentence = df.iloc[4000]['en']
fr_sentence = df.iloc[4000]['fr']

In [15]:
inputs = tokenizer(en_sentence)

In [16]:
inputs

{'input_ids': [488, 26702, 117, 1249, 1459, 26, 7081, 39193, 324, 402, 18335, 13169, 50, 57, 638, 5816, 2, 55, 115, 256, 4, 47009, 16488, 50, 992, 3036, 55, 115, 760, 4, 402, 180, 7828, 50, 1437, 1097, 117, 11949, 760, 108, 42, 16488, 50, 1808, 15958, 4, 402, 180, 7828, 50, 12, 15, 10139, 1437, 42, 16488, 50, 2465, 10, 760, 4, 10139, 1437, 42, 16488, 50, 2042, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
fr_sentence

"Pour compiler votre code source pour un afficheur à l'écran avec & okular; ou pour d'autres conversions, vous pouvez utiliser le raccourci Alt; 2. Vous pouvez alors voir le fichier & DVI; en utilisant votre afficheur par défaut avec Alt; 3, convertir le & DVI; en fichier PS avec Alt; 4, et voir le fichier PS avec Alt; 5."

In [18]:
with tokenizer.as_target_tokenizer():
    targets = tokenizer(fr_sentence)

C:\Users\niels\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [19]:
print(fr_sentence)

Pour compiler votre code source pour un afficheur à l'écran avec & okular; ou pour d'autres conversions, vous pouvez utiliser le raccourci Alt; 2. Vous pouvez alors voir le fichier & DVI; en utilisant votre afficheur par défaut avec Alt; 3, convertir le & DVI; en fichier PS avec Alt; 4, et voir le fichier PS avec Alt; 5.


In [20]:
max_input_length = 128
max_target_length = 128

In [21]:
def preprocess_function(examples):
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['fr'] for ex in examples['translation']]
    
    model_inputs = tokenizer(inputs,max_length = max_input_length,truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets,max_length = max_target_length,truncation=True)
        
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

In [22]:
tokenized_datasets = split_datasets.map(preprocess_function,
                                       batched=True,
                                       remove_columns=split_datasets['train'].column_names)

Loading cached processed dataset at C:\Users\niels\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-d821afeb12e2c061.arrow
Loading cached processed dataset at C:\Users\niels\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-f58d27b38d139f31.arrow


In [23]:
tokenized_datasets['train'][9]

{'input_ids': [1590, 4490, 323, 0],
 'attention_mask': [1, 1, 1, 1],
 'labels': [1590, 4490, 323, 2061, 21715, 738, 0]}

In [24]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})

In [25]:
from transformers import AutoModelForSeq2SeqLM

In [26]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [27]:
from transformers import DataCollatorForSeq2Seq

In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)

In [29]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])

In [30]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [31]:
batch['labels']

tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]])

In [32]:
batch['decoder_input_ids']

tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])

In [33]:
import evaluate

In [34]:
import sacrebleu

In [35]:
metric = evaluate.load('sacrebleu')

In [36]:
import numpy as np

In [37]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds,tuple):
        preds = preds[0]
        
    decoded_preds = tokenizer.batch_decode(preds,skip_special_tokens=True)
    
    labels = np.where(labels != -100,labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels,skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds,references=decoded_labels)
    
    return {'bleu': result['score']}

In [38]:
from transformers import Seq2SeqTrainingArguments

In [39]:
args = Seq2SeqTrainingArguments(f'marian-finetuned-kde4-en-to-fr',
                               evaluation_strategy='no',
                               save_strategy='epoch',
                               learning_rate=2e-5,
                               per_device_train_batch_size=32,
                               per_device_eval_batch_size=64,
                               weight_decay=0.01,
                               save_total_limit=3,
                               predict_with_generate=True,
                               fp16=True,
                               push_to_hub=True)

In [40]:
from huggingface_hub import notebook_login

In [44]:
notebook_login()


In [45]:
from transformers import Seq2SeqTrainer

In [46]:
trainer = Seq2SeqTrainer(model,
                         args,
                        train_dataset=tokenized_datasets['train'],
                        eval_dataset=tokenized_datasets['test'],
                        data_collator=data_collator,
                        tokenizer=tokenizer,
                        compute_metrics=compute_metrics)

In [47]:
trainer.evaluate(max_length=max_target_length)

{'eval_loss': 1.6964490413665771,
 'eval_bleu': 39.162398889068854,
 'eval_runtime': 30792.8908,
 'eval_samples_per_second': 0.683,
 'eval_steps_per_second': 0.011}

In [48]:
trainer.train()

Step,Training Loss
500,1.423600
1000,1.230000
1500,1.186100
2000,1.125100
2500,1.112500
3000,1.074600
3500,1.064400
4000,1.034000
4500,1.019400
5000,1.009600


TrainOutput(global_step=17736, training_loss=0.9378722650702601, metrics={'train_runtime': 92967.4168, 'train_samples_per_second': 6.104, 'train_steps_per_second': 0.191, 'total_flos': 1.1305306504691712e+16, 'train_loss': 0.9378722650702601, 'epoch': 3.0})

In [106]:
text = 'Generative models are the future'

In [107]:
inputs = tokenizer(text, return_tensors="pt").input_ids.cuda()

In [108]:
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [109]:
outputs

tensor([[59513,    83,  3693, 12852,   108, 13419,    70,    19,  7034,   657,
         23691,  1510, 10749,   323,  3644, 18467,     0]], device='cuda:0')

In [110]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Les modèles generatifs sont le futurNAME OF TRANSLATORS'